In [30]:
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd
#pd.set_option('display.max_rows', 500)
#pd.set_option('display.max_columns', 500)
#pd.set_option('display.width', 1000)

import matplotlib.pyplot as plt
import seaborn as sns
import scipy
from scipy.stats import probplot


from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

import warnings
warnings.filterwarnings('ignore')

## Optiver Realized Volatility Prediction

## 1. Introduction

This competition's objective is predicting short-term volatility for 112 stocks across different sectors. Dataset consist book and trade data of the stocks for multiple time buckets. We are supposed to predict a target value (volatility) for every time bucket of the stocks. 107 of the stocks have 3830, 3 of the stocks have 3829, 1 of the stocks has 3820 and 1 of the stocks has 3815 time buckets in training dataset. Thus, there are 428,932 rows to predict in it. Since this is a code competition, public test set is hidden and private test set will be real market data collected in the three-month evaluation period after competition ends. There are 3 columns in training set and 2 columns in placeholder test set.

* `stock_id` - ID of the stock
* `time_id` - ID of the time bucket
* `target` - Realized volatility of the next 10 minute window under the same stock_id/time_id (doesn't exist in placeholder test set)

In [31]:
train_test_dtypes = {
    'stock_id': np.uint8,
    'time_id': np.uint16,
    'target': np.float64
}

df_train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv', dtype=train_test_dtypes)
df_test = pd.read_csv('../input/optiver-realized-volatility-prediction/test.csv', usecols=['stock_id', 'time_id'], dtype=train_test_dtypes)

print(f'Training Set Shape: {df_train.shape}')
print(f'Training Set Memory Usage: {df_train.memory_usage().sum() / 1024 ** 2:.2f} MB')
print(f'Test Set Shape: {df_test.shape}')
print(f'Test Set Memory Usage: {df_test.memory_usage().sum() / 1024 ** 2:.2f} MB')

In [32]:
df_train

## 2. Evaluation

Submissions are scored using RMSPE (root mean squared percentage error) which can be denoted as

$\huge \text{RMSPE} = \sqrt{\frac{1}{n} \sum_{i=1}^{n} ((y_i - \hat{y}_i)/y_i)^2}$

RMSPE is very similar to RMSE. The only difference between them is, the error is divided by the actual value. Predictions closer to actual values yield errors closer to 0, so division by actual values is sensitive to larger errors. In addition to that, errors are squared before they are averaged, which makes this metric even more sensitive to larger errors. This means larger errors are not tolerable in this domain.

One pitfall of RMSPE is it can raise ZeroDivisionError if a single data point in actual values is equal to 0. Even though, there isn't any 0 target values in training set, this can be easily solved by adding a small constant to actual values. A small constant epsilon wouldn't contribute to overall rmspe and it prevents ZeroDivisionError.

For baselines, global target mean scores **1.11033**, target mean of stock_ids scores **0.789618**, and target median of stock_ids scores **0.589135**  on training set.

In [33]:
def root_mean_squared_percentage_error(y_true, y_pred, epsilon=1e-10):
    
    rmspe = np.sqrt(np.mean(np.square((y_true - y_pred) / (y_true + epsilon))))
    return rmspe


target_mean_rmspe = root_mean_squared_percentage_error(df_train['target'], np.repeat(df_train['target'].mean(), len(df_train)))
print(f'target Mean RMPSE: {target_mean_rmspe:.6}')

stock_id_target_mean_rmspe = root_mean_squared_percentage_error(df_train['target'], df_train.groupby('stock_id')['target'].transform('mean'))
print(f'stock_id target Mean RMPSE: {stock_id_target_mean_rmspe:.6}')

stock_id_target_median_rmspe = root_mean_squared_percentage_error(df_train['target'], df_train.groupby('stock_id')['target'].transform('median'))
print(f'stock_id target Median RMPSE: {stock_id_target_median_rmspe:.6}')

## 3. Realized Volatility (Target)

Target is defined as realized volatility computed over the 10 minute window following the feature data under the same stock/time_id. For every stock_id and its time_id, target is the next 10 minute realized volatility of the same stock_id and time_id. Thus, there are 10 minute gaps between provided dataset and target values, and there is no overlap between them.

Realized volatility, $\sigma$, is the squared root of the sum of squared log returns which is denoted as

$\huge \sigma = \sqrt{\sum_{t}r_{t-1, t}^2}$

and log return of a stock $S$ at time $t$ is

$\huge r_{t-1, t} = \log \left( \frac{S_{t-1}}{S_{t1}} \right)$

where $S_t$ is the price of the stock $S$ at time $t$. The price used for targets is the weighted averaged price (WAP) and it can be derived from book data. Log returns are used for realized volatility calculation because price differences are not always comparable across stocks.

In [34]:
def visualize_target(target, df_train):
    
    print(f'{target}\n{"-" * len(target)}')
        
    print(f'Mean: {df_train[target].mean():.4f}  -  Median: {df_train[target].median():.4f}  -  Std: {df_train[target].std():.4f}')
    print(f'Min: {df_train[target].min():.4f}  -  25%: {df_train[target].quantile(0.25):.4f}  -  50%: {df_train[target].quantile(0.5):.4f}  -  75%: {df_train[target].quantile(0.75):.4f}  -  Max: {df_train[target].max():.4f}')
    print(f'Skew: {df_train[target].skew():.4f}  -  Kurtosis: {df_train[target].kurtosis():.4f}')
    missing_values_count = df_train[df_train[target].isnull()].shape[0]
    training_samples_count = df_train.shape[0]
    print(f'Missing Values: {missing_values_count}/{training_samples_count} ({missing_values_count * 100 / training_samples_count:.4f}%)')

    fig, axes = plt.subplots(ncols=1, figsize=(12, 8), dpi=100)
    sns.distplot(df_train[target], label=target, kde=True, ax=axes)
    axes.axvline(df_train[target].mean(), label=f'{target} Mean', color='r', linewidth=2, linestyle='--')
    axes.axvline(df_train[target].median(), label=f'{target} Median', color='b', linewidth=2, linestyle='--')
    axes.legend(prop={'size': 16})
    

    axes.set_title(f'{target} Distribution in Training Set', fontsize=20, pad=15)
    

    plt.show()


In [35]:
visualize_target('target',df_train)

* Distribution of target for specific stocks.

In [36]:
visualize_target('target',df_train[df_train['stock_id']==0])

In [37]:
visualize_target('target',df_train[df_train['stock_id']==37])

* Very similar to a Log normal: $\frac{1}{\sqrt{2\pi}x\sigma} e^{-\frac{(\ln x -\mu)^2}{2\sigma^2}}$

In [38]:
# log normal
x = np.arange(0.01,10,0.01)
def log_normal(x, mu, std):
    return np.exp(-(np.log(x)-mu)**2 /(2*std**2) ) / (x*std*np.sqrt(2*np.pi))
mu=0; std=1
sample = log_normal(x,mu,std)

axes = plt.subplots(ncols=1, figsize=(12, 8), dpi=100)
axes = plt.plot(x,sample)
plt.axvline(np.exp(mu+std**2/2), label=f'Mean', color='r', linewidth=2, linestyle='--')
plt.axvline(np.exp(mu), label=f'Median', color='b', linewidth=2, linestyle='--')
plt.show()

* Fréchet distribution, since at each day, the WAP is derived from book price that are maximum bet and minimum ask price. It might have a extreme value distribution. Fréchet distribution density: $\frac{\alpha}{s} \; \left(\frac{x-m}{s}\right)^{-1-\alpha} \; e^{-(\frac{x-m}{s})^{-\alpha}}$

* It is no where close to a chi distribution

In [39]:
# sample = 0.001*np.sqrt(np.random.noncentral_chisquare(1000,4,600))

# axes = plt.subplots(ncols=1, figsize=(12, 8), dpi=100)
# axes= sns.histplot(sample, kde=True)
# #
# axes.axvline(sample.mean(), label=f'Mean', color='r', linewidth=2, linestyle='--')
# axes.axvline(np.median(sample), label=f'Median', color='b', linewidth=2, linestyle='--')
# axes.legend(prop={'size': 16})
# plt.show()

Each stock is very different in terms of volatility. Stocks are displayed below from most volatile to least volatile. They are ranked by their mean target value across multiple time_ids. Wide error bars show that even the realized volatility is volatile between multiple time buckets. One stock can be extremely volatile in one time bucket and less volatile in another time bucket. This phenomenon can be explained by time_ids not being sequential, and there aren't any temporal dependencies between them.

In [40]:
df_train.groupby('stock_id')['target']

In [41]:
target_means = df_train.groupby('stock_id')['target'].mean()
target_stds = df_train.groupby('stock_id')['target'].std()

target_means_and_stds = pd.concat([target_means, target_stds], axis=1)
target_means_and_stds.columns = ['mean', 'std']
target_means_and_stds.sort_values(by='mean', ascending=True, inplace=True)

fig, ax = plt.subplots(figsize=(32, 48))
ax.barh(
    y=np.arange(len(target_means_and_stds)),
    width=target_means_and_stds['mean'],
    xerr=target_means_and_stds['std'],
    align='center',
    ecolor='black',
    capsize=3
)

ax.set_yticks(np.arange(len(target_means_and_stds)))
ax.set_yticklabels(target_means_and_stds.index)
ax.set_xlabel('target', size=20, labelpad=15)
ax.set_ylabel('stock_id', size=20, labelpad=15)
ax.tick_params(axis='x', labelsize=20, pad=10)
ax.tick_params(axis='y', labelsize=20, pad=10)
ax.set_title('Mean Realized Volatility of Stocks', size=25, pad=20)

plt.show()

del target_means, target_stds, target_means_and_stds

Instead of entire stocks, individual time buckets from different stocks are ranked based on their realized volatility. 
* The most volatile 10 time buckets: The most volatile time bucket belongs to stock 77 and its time_id is 24600. The most volatile stock was stock 18 and it has 3 time buckets in this list.

* The least volatile 10 time buckets: All of the least volatile 10 time buckets belong to stock 31, even though it has an average volatility overall. This could be an anomaly and it must be explored further.

In [42]:
df_train['stock_time_id'] = df_train['stock_id'].astype(str) + '_' + df_train['time_id'].astype(str)

#Top 10 Most Volatile Time Buckets
fig, ax = plt.subplots(figsize=(32, 10))
ax.barh(
    y=np.arange(10),
    width=df_train.sort_values(by='target', ascending=True).tail(10)['target'],
    align='center',
    ecolor='black',
)

ax.set_yticks(np.arange(10))
ax.set_yticklabels(df_train.sort_values(by='target', ascending=True).tail(10)['stock_time_id'])
ax.set_xlabel('target', size=20, labelpad=15)
ax.set_ylabel('stock_time_id', size=20, labelpad=15)
ax.tick_params(axis='x', labelsize=20, pad=10)
ax.tick_params(axis='y', labelsize=20, pad=10)
ax.set_title('Top 10 Most Volatile Time Buckets', size=25, pad=20)

plt.show()

# Top 10 Least Volatile Time Buckets
fig, ax = plt.subplots(figsize=(32, 10))
ax.barh(
    y=np.arange(10),
    width=df_train.sort_values(by='target', ascending=True).head(10)['target'],
    align='center',
    ecolor='black',
)

ax.set_yticks(np.arange(10))
ax.set_yticklabels(df_train.sort_values(by='target', ascending=True).head(10)['stock_time_id'])
ax.set_xlabel('target', size=20, labelpad=15)
ax.set_ylabel('stock_time_id', size=20, labelpad=15)
ax.tick_params(axis='x', labelsize=20, pad=10)
ax.tick_params(axis='y', labelsize=20, pad=10)
ax.set_title('Top 10 Least Volatile Time Buckets', size=25, pad=20)

plt.show()

df_train.drop(columns=['stock_time_id'], inplace=True)

### Correlations of target is very high
stock_id 12 is not present;
stock_id 13,38,75,80,100 have different length.

In [43]:
df_train_tar =pd.DataFrame(index=df_train.time_id.unique())
for i in df_train.stock_id.unique():
    try:
        df_train_tar[i] = df_train[df_train['stock_id']==i]['target'].values
    except ValueError as e:
        print(i, e)

In [44]:
def corr_matrix(df,threshold=0.4):
    #we color correlation > threshold
    cor = df.corr().round(2)
    np.fill_diagonal(cor.values, 0)
    Bigcor = (cor.abs()> threshold ).any(1)[(cor.abs()> threshold ).any(1)==True].index
    cor =(df[Bigcor]).corr().round(2).abs()
    # generate mask to filter out any number < threshold
    mask = np.triu(np.ones_like(cor, dtype=np.bool))+ np.array(cor.abs()<threshold)
    sns.set(font_scale=0.8)
    plt.figure(figsize=(19, 15))
    sns.heatmap(cor, annot=True, fmt='.1f', cmap='coolwarm', mask=mask, linewidths=1, cbar=True)
    plt.xticks(rotation=30, ha="right") 
    plt.title('Linear correlaton', fontsize=16)
    plt.show()
    
corr_matrix(df_train_tar.iloc[:,10:30],threshold=0.4)

* use index to combine those columns with shorter time_id's.

In [45]:
df_train_tar = pd.DataFrame(index=df_train.time_id.unique())
for i in df_train.stock_id.unique():
    try:
        df_train_tar[i] = df_train[df_train['stock_id']==i][['time_id','target']].set_index('time_id')
    except ValueError as e:
        print(i, e)             


In [46]:
df_train_tar[df_train_tar[13].isnull()][13]

In [47]:
corr_matrix(df_train_tar.iloc[:,65:90],threshold=0.4)

In [48]:
plt.scatter(df_train_tar[0], df_train_tar[1])
plt.show()

In [49]:
def column_to_time_stock_matrix(df, col_name):
    # turn a column df_train[col_name] to a time * stocks matrix
    return df.pivot(index='time_id', columns='stock_id', values= col_name)

def corr_matrix(df, threshold=0.4, drop_low = False):
    #we only keep correlation > threshold
    cor =df.corr().round(2)
    if drop_low: # drop columns that have all correlations < threshold
        large_col = (cor.abs()> threshold ).any(1)[(cor.abs()> threshold ).any(1)==True].index
        cor =(df[large_col]).corr().round(2).abs()
    return cor
def see_matrix(cor,threshold=0.4):
    # generate mask to filter out any number < threshold
    np.fill_diagonal(cor.values, 0)
    mask = np.triu(np.ones_like(cor, dtype=np.bool))+ np.array(cor.abs()<threshold)
    sns.set(font_scale=0.8)
    fig = plt.figure(figsize=(19, 15))
    sns.heatmap(cor, annot=True, fmt='.2f', cmap='coolwarm', mask=mask, linewidths=1, cbar=True)
    plt.xticks(rotation=30, ha="right") 
    #plt.show()
    np.fill_diagonal(cor.values, 1)


# turn target and prediction columns to matrix
# df_reVol =  column_to_time_stock_matrix(test, 'log_return1_sqrt_sum_square')
df_tar = column_to_time_stock_matrix(df_train, 'target')


# Stock correlation
corr =corr_matrix(df_tar)   
see_matrix(corr,threshold=0.8)


* correlation - Clustering

In [50]:
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from scipy.spatial.distance import squareform


dissimilarity = 1 - abs(df_train_tar.corr())
Z = linkage(squareform(dissimilarity), 'complete')



In [51]:
plt.figure(figsize=(20,10))
dendrogram(Z, labels=df_train_tar.columns, orientation='top', 
           leaf_rotation=90);
ax = plt.gca()
ax.tick_params(axis='x', which='major', labelsize=12)

## 4. Order Book

Order book is a list of buy and sell orders organized by price level for given stocks. An order book lists the number of shares being bid on or asked at each price point. Order books help to improve market transparency as they provide information on price, availability and depth of trade.

Order book data files are named as `book_train.parquet` and `book_test.parquet`, and they are parquet files partitioned by stock_id column. Partitioned by stock_id means those files can both read as whole or individual stocks. However, reading them as a single file is not easy as they will consume too much memory.

There are 10 columns in every book data partition. The columns are:

* `time_id` - ID of the time bucket
* `seconds_in_bucket` - Number of seconds passed since the start of the bucket
* `bid_price1` - Highest buy price after normalization
* `ask_price1` - Lowest sell price after normalization
* `bid_price2` - Second highest buy price after normalization
* `ask_price2` - Second lowest sell price after normalization
* `bid_size1` - Number of shares on the highest buy price
* `ask_size1` - Number of shares on the lowest sell price
* `bid_size2` - Number of shares on the second highest buy price
* `ask_size2` - Number of shares on the second lowest sell price

Values in order book are the last snapshots of each second. Some of the seconds are not available because there weren't any related market activities during those seconds, thus order books are not updated. Normally, given order books last 600 seconds for every time bucket. Values of the missing seconds are the values at the last updated second, so the book data can be reindexed to 600 seconds for every time bucket and missing values can be forward filled for every field. Forward filling and sorting order books by time_id and seconds_in_bucket doesn't change the extracted features but these functionalities can be added for private test set as a sanity check.

As mentioned, realized volatilities are calculated from the weighted averaged price in order books of every stock. The formula for weighted averaged price is

$\large WAP = \frac{BidPrice_{1}*AskSize_{1} + AskPrice_{1}*BidSize_{1}}{BidSize_{1} + AskSize_{1}}$

Realized volatilities are calculated using the most competitive buy and sell levels, but same formula can applied to second most competitive buy and sell levels or other prices/sizes as well. After that, log returns of the weighted averaged price are summed and square rooted for realized volatility calculation.

Realized volatility, $\sigma$, is the squared root of the sum of squared log returns which is denoted as $ \sigma = \sqrt{\sum_{t}r_{t-1, t}^2}$and log return of a stock $S$ at time $t$ is $ r_{t-1, t} = \log \left( \frac{S_{t-1}}{S_{t1}} \right)$  where $S_t$ is the price of the stock $S$ at time $t$.

In [52]:
def read_book_data(dataset, stock_id, short=False, sort=True, forward_fill=False):
    book_dtypes = {
        'time_id': np.uint16,
        'seconds_in_bucket': np.uint16,
        'bid_price1': np.float32,
        'ask_price1': np.float32,
        'bid_size1': np.uint32,
        'ask_size1': np.uint32,
    }
    if not short: # want more information
        book_dtypes= {**book_dtypes,
                        'bid_price2': np.float32,
                        'ask_price2': np.float32,
                        'bid_size2': np.uint32,
                        'ask_size2': np.uint32,}

    df_book = pd.read_parquet(f'../input/optiver-realized-volatility-prediction/book_{dataset}.parquet/stock_id={stock_id}',
                              columns = book_dtypes.keys())

    for column, dtype in book_dtypes.items():
        df_book[column] = df_book[column].astype(dtype)
    
    if sort:
        df_book.sort_values(by=['time_id', 'seconds_in_bucket'], inplace=True)
        
    if forward_fill:
        df_book = df_book.set_index(['time_id', 'seconds_in_bucket'])
        df_book = df_book.reindex(pd.MultiIndex.from_product([df_book.index.levels[0], np.arange(0, 600)], names=['time_id', 'seconds_in_bucket']), method='ffill')
        df_book.reset_index(inplace=True)

    return df_book

# read_book_data('train', 1,short=True)

In [53]:
read_book_data('train', 1,short=True)

In [54]:
def save_log_return(df_train):
    for stock_id in tqdm(sorted(df_train['stock_id'].unique())):

        df_book = read_book_data('train', stock_id)

        # Weighted averaged prices
        df_book['wap1'] = (df_book['bid_price1'] * df_book['ask_size1'] + df_book['ask_price1'] * df_book['bid_size1']) /\
                          (df_book['bid_size1'] + df_book['ask_size1']).astype('float32')
        df_book['wap2'] = (df_book['bid_price2'] * df_book['ask_size2'] + df_book['ask_price2'] * df_book['bid_size2']) /\
                          (df_book['bid_size2'] + df_book['ask_size2']).astype('float32')

        # Realized volatilities rv1, rv2
        for wap in [1, 2]:
            df_book[f'squared_log_return_from_wap{wap}'] = (np.log(df_book[f'wap{wap}'] / df_book.groupby('time_id')[f'wap{wap}'].shift(1)) ** 2).astype('float32')
             # save data for deeper exploration
        df_book[['time_id','seconds_in_bucket',f'squared_log_return_from_wap1',f'squared_log_return_from_wap2']].to_parquet(f'log_return_{stock_id}.parquet')
    return 0
SAVED = True
if not SAVED:
    save_log_return(df_train)

In [55]:
# this will use the log_returns saved in above function to generate features as columns in df_train
def feature_from_log_return(df_train, old_fea =True, split=5):
    
    for stock_id in tqdm(sorted(df_train['stock_id'].unique())):
        df_book = pd.read_parquet(f'../input/optiver-log-return/log_return_{stock_id}.parquet') # (f'log_return_{stock_id}.parquet')
        
        if old_fea: # put old feature under if, so it will not be generated repeatedly in next run
            # Realized volatilities features rv1 from wap1, rv2 from wap2
            for wap in [1, 2]:
                realized_volatilities = np.sqrt(df_book.groupby('time_id')[f'squared_log_return_from_wap{wap}'].sum()).to_dict()  
                df_train.loc[df_train['stock_id'] == stock_id, f'rv{wap}']= df_train[df_train['stock_id'] == stock_id]['time_id'].map(realized_volatilities).astype('float32')
        
        # realized volatilities from only wap1, but split 
        # at 0_split (time < 60* split)
        realized_volatilities = np.sqrt(df_book[df_book['seconds_in_bucket']<60*split].groupby('time_id')['squared_log_return_from_wap1'].sum()).to_dict()
        df_train.loc[df_train['stock_id'] == stock_id, f'rv_0_{split}']= df_train[df_train['stock_id'] == stock_id]['time_id'].map(realized_volatilities).astype('float32')
        # at 0_split (time >= 60* split)
        realized_volatilities = np.sqrt(df_book[df_book['seconds_in_bucket']>=60*split].groupby('time_id')['squared_log_return_from_wap1'].sum()).to_dict()
        df_train.loc[df_train['stock_id'] == stock_id, f'rv_{split}_10']= df_train[df_train['stock_id'] == stock_id]['time_id'].map(realized_volatilities).astype('float32')
    
    # save df_train
    df_train.to_parquet('train.parquet')
    return df_train
SAVED = True
if SAVED: 
    df_train = pd.read_parquet('../input/optiver-log-return/train.parquet')
else:
    df_train = feature_from_log_return(df_train, old_fea = True, split=5)

In [56]:
df_train.head()

In [57]:
def feature_error(feature):
    RMSPE = root_mean_squared_percentage_error(df_train['target'], df_train[feature])
    print(f'{feature} RMPSE: {RMSPE:.6}')

# Realized volatilities calculated from WAP1 scores **0.341354** 
feature_error('rv1')
# Realized volatilities calculated from WAP2 scores **0.705453** 
feature_error('rv2')
# Realized volatilities calculated from WAP1 0-5mins scores **0.347109** 
feature_error('rv_0_5')
# Realized volatilities calculated from WAP1 5-10mins scores **0.340067** 
feature_error('rv_5_10')

## correlations between features and target

In [58]:
def column_to_time_stock_matrix(df_train, col_name):
    # turn a column df_train[col_name] to a time * stocks matrix
    df = pd.DataFrame(index=df_train.time_id.unique())
    for i in df_train.stock_id.unique():
        try:
            df[i] = df_train[df_train['stock_id']==i][['time_id',col_name]].set_index('time_id')
        except ValueError as e:
            print(i, e)  
    return df

In [59]:
# turn target column to matrix
df_tar = column_to_time_stock_matrix(df_train, 'target')

# turn feature columns to matrix 
fea_col = df_train.columns[3:]
print('features: ', fea_col)
features = {}
df_corr= pd.DataFrame()
for feat in fea_col:
    features[feat] = column_to_time_stock_matrix(df_train, feat)
    df_corr['targ_'+feat] = df_tar.corrwith(features[feat])
sns.displot(df_corr, kde=True)
plt.show()

## correlations between different stocks for each feature

In [60]:
def corr_matrix(df, threshold=0.4, drop_low = False):
    #we only keep correlation > threshold
    cor =df.corr().round(2)
    if drop_low: # drop columns that have all correlations < threshold
        large_col = (cor.abs()> threshold ).any(1)[(cor.abs()> threshold ).any(1)==True].index
        cor =(df[large_col]).corr().round(2).abs()
    return cor
def see_matrix(cor,threshold=0.4):
    # generate mask to filter out any number < threshold
    np.fill_diagonal(cor.values, 0)
    mask = np.triu(np.ones_like(cor, dtype=np.bool))+ np.array(cor.abs()<threshold)
    sns.set(font_scale=0.8)
    plt.figure(figsize=(19, 15))
    sns.heatmap(cor, annot=True, fmt='.2f', cmap='coolwarm', mask=mask, linewidths=1, cbar=True)
    plt.xticks(rotation=30, ha="right") 
    plt.title('Linear correlaton', fontsize=16)
    plt.show()
    np.fill_diagonal(cor.values, 1)

corr =corr_matrix(df_tar.loc[:,:])   
see_matrix(corr,threshold=0.8)

In [61]:
plt.figure(figsize=(20,10))
dendrogram(Z, labels=df_train_tar.columns, orientation='top', 
           leaf_rotation=90);
ax = plt.gca()
ax.tick_params(axis='x', which='major', labelsize=12)

In [62]:
corr[[3,8,17,20,40,80,81,95,102,110,123]]
#corr[123].sort_values(ascending=True)
#corr[8].sort_values(ascending=True)

- Difference of correlation matrix is small

In [63]:
see_matrix(features['rv_5_10'].corr()-df_tar.corr(),0.05)

The order book helps traders to make more informed trading decisions by showing order imbalances that may provide clues to a stock’s direction in the very short term. A huge imbalance of buy orders against sell orders may indicate a move higher in the stock due to buying pressure, or vice versa. Traders can also use the order book to help pinpoint a stock’s potential support and resistance levels. A cluster of large buy orders at a specific price may indicate a level of support, while an abundance of sell orders at or near one price may suggest an area of resistance.
Realized volatilities increase when those moves in either directions become more frequent. The function in the cell below is going to be used for visualizing individual order book time buckets and trying to find clues about their volatilities.

In [64]:
def visualize_book_time_bucket(stock_id, time_id):
    
    time_bucket = (df_train['stock_id'] == stock_id) & (df_train['time_id'] == time_id)
    
    target = df_train.loc[time_bucket, 'target'].iloc[0]
    realized_volatility = df_train.loc[time_bucket, 'rv1'].iloc[0]
    df_book = read_book_data('train', stock_id, sort=True, forward_fill=True)
    df_book = df_book.set_index('seconds_in_bucket')
    
    df_book['wap1'] = (df_book['bid_price1'] * df_book['ask_size1'] + df_book['ask_price1'] * df_book['bid_size1']) /\
                      (df_book['bid_size1'] + df_book['ask_size1'])
    df_book['wap2'] = (df_book['bid_price2'] * df_book['ask_size2'] + df_book['ask_price2'] * df_book['bid_size2']) /\
                      (df_book['bid_size2'] + df_book['ask_size2'])
    
    fig, axes = plt.subplots(figsize=(32, 30), nrows=2)
    
    axes[0].plot(df_book.loc[df_book['time_id'] == time_id, 'bid_price1'], label='bid_price1', lw=2, color='tab:green')
    axes[0].plot(df_book.loc[df_book['time_id'] == time_id, 'ask_price1'], label='ask_price1', lw=2, color='tab:red')
    axes[0].plot(df_book.loc[df_book['time_id'] == time_id, 'bid_price2'], label='bid_price2', alpha=0.3, color='tab:green')
    axes[0].plot(df_book.loc[df_book['time_id'] == time_id, 'ask_price2'], label='ask_price2', alpha=0.3, color='tab:red')
    axes[0].plot(df_book.loc[df_book['time_id'] == time_id, 'wap1'], label='wap1', lw=2, linestyle='--', color='tab:blue')
    axes[0].plot(df_book.loc[df_book['time_id'] == time_id, 'wap2'], label='wap2', alpha=0.3, linestyle='--',  color='tab:blue')
    
    axes[1].plot(df_book.loc[df_book['time_id'] == time_id, 'bid_size1'], label='bid_size1', lw=2, color='tab:green')
    axes[1].plot(df_book.loc[df_book['time_id'] == time_id, 'ask_size1'], label='ask_size1', lw=2, color='tab:red')
    axes[1].plot(df_book.loc[df_book['time_id'] == time_id, 'bid_size2'], label='bid_size2', alpha=0.3, color='tab:green')
    axes[1].plot(df_book.loc[df_book['time_id'] == time_id, 'ask_size2'], label='ask_size2', alpha=0.3, color='tab:red')
    
    for i in range(2):
        axes[i].legend(prop={'size': 18})
        axes[i].tick_params(axis='x', labelsize=20, pad=10)
        axes[i].tick_params(axis='y', labelsize=20, pad=10)
    axes[0].set_ylabel('price', size=20, labelpad=15)
    axes[1].set_ylabel('size', size=20, labelpad=15)
    
    axes[0].set_title(
        f'Prices of stock_id {stock_id} time_id {time_id} - Current Realized Volatility: {realized_volatility:.6f} - Next 10 minute Realized Volatility: {target:.6f}',
        size=25,
        pad=15
    )
    axes[1].set_title(
        f'Sizes of stock_id {stock_id} time_id {time_id} - Current Realized Volatility: {realized_volatility:.6f} - Next 10 minute Realized Volatility: {target:.6f}',
        size=25,
        pad=15
    )
    
    plt.show()


The most volatile time bucket in next 10 minute window is time **24600** from stock **77**. Current realized volatility of that time bucket is 0.02 which is roughly greater than 99.7% of other current realized volatilities. The time bucket was extremely volatile in current 10 minute window and it became the most volatile time bucket in next 10 minute window. Short-term realized volatilities are definitely correlated in this case. First of all, this time bucket is on a decreasing trend because large amount of most competitive ask sizes are always dominating most competitive bid sizes. Some traders were trying to sell lots of shares at most competitive price very frequently. In addition to that, this time bucket has more updates compared to others. Update frequency of time buckets might be correlated to realized volatility since more squared log return values greater than 0 will be summed.

In [65]:
visualize_book_time_bucket(stock_id=77, time_id=24600)

The most volatile time bucket in current 10 minute window is time **30128** from stock **30**. Realized volatility of the next 10 minute window for this time bucket is 0.039 which is roughly greater than %99.994 of other target values. In this case, this time bucket became less volatile in next 10 minute window and that suggests it is really hard to tell whether the realized volatility will be more or less in short-term, but it is expected to be closer to current realized volatility. This stock was extremely volatile in this time bucket because large amount of sell orders at most competitive prize dominated buy orders in first quarter and the opposite happened in second quarter. Later, buy/sell order domination kept switching places in smaller scales and that caused smaller spikes in weigted averaged price 1.

In [66]:
visualize_book_time_bucket(stock_id=30, time_id=30128)

The least volatile time bucket in next 10 minute window is time **8534** from stock **31**. The sizes of most competitive bid and ask orders were always balanced until the very end. Couple changes at the end caused a small decrease and increase in weighted average price 1. That was the only significant contribution to current realized volatility. This time bucket became almost 20 times less volatile in next 10 minute window.

In [67]:
visualize_book_time_bucket(stock_id=31, time_id=8534)

The least volatile time bucket in current 10 minute window is time **28959** from stock **31**. Prices and sizes of most competitive bid and ask orders are exteremely balanced in this entire time bucket. Sizes of most competitive bid level is shifted to a new state at the second half. That movement slightly increased the weighted averaged price 1, and that was the only thing contributed to current realized volatility. Stock 31 is very calm compared to other stocks. Sizes of most competitive bid and ask levels could be the reason of this phenomenon, or it could be an anomaly.

In [68]:
visualize_book_time_bucket(stock_id=31, time_id=28959)

To wrap up, order book is very important as it gives information about buy and sell orders at most competitive levels. Target is also derived from order books of the stocks, so features extracted from them will be very useful. Features that are capturing the price/size imbalances might be useful for models to predict realized volatilities of next 10 minute window.

In [69]:
def get_book(stock_id):
    df = read_book_data('train', stock_id, sort=True, forward_fill=True)
    
    df['wap1'] = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) /\
                      (df['bid_size1'] + df['ask_size1'])
    
    df['RV'] = df.groupby(['time_id'])['wap1'].apply(lambda x: np.log(x).diff().pow(2,fill_value=0).cumsum().pow(0.5) )
    return df[['time_id', 'seconds_in_bucket','RV']]

def visualize_book_time_bucket(df_book, stock_id, time_id):
    
    time_bucket = (df_train['stock_id'] == stock_id) & (df_train['time_id'] == time_id)
    
    target = df_train.loc[time_bucket, 'target'].iloc[0]
    realized_volatility = df_train.loc[time_bucket, 'rv1'].iloc[0]
    plt.plot(df_book.loc[df_book['time_id'] == time_id, 'wap1'], label='wap1', lw=2, linestyle='--', color='tab:blue')
    plt.plot(df_book.loc[df_book['time_id'] == time_id, 'wap2'], label='wap2', alpha=0.3, linestyle='--',  color='tab:blue')

    plt.title(
        f'Prices of stock_id {stock_id} time_id {time_id} - Current Realized Volatility: {realized_volatility:.6f} - Next 10 mins Realized Volatility: {target:.6f}',
    )
    
    plt.show()


In [70]:
df_new = df_train.drop(['rv1', 'rv2', 'rv_0_5', 'rv_5_10'],axis=1)
df_new['seconds_in_bucket']=600
df_new.rename(columns={'target': 'RV'}, inplace=True)

stock_id = 0
df_book = get_book(stock_id)
df_target = df_new[df_new['stock_id']== stock_id].drop('stock_id',axis=1)
df = pd.concat([df_book, df_target], ignore_index=True)

In [71]:
df

In [72]:
plt.figure(figsize=(20,15))
for time_id in df_train['time_id'].unique()[40:50]:
    plt.plot(df.loc[df['time_id'] == time_id, 'seconds_in_bucket'], 
                df.loc[df['time_id'] == time_id, 'RV'])
    
    #visualize_book_time_bucket(df_book, stock_id=stock_id, time_id=time_id)

## 4.1 Model 
$X \beta = Y$, where $X: t \times k$, $\beta: k\times k$ and $Y: t \times k$. $k$ is the number of stocks, $t$ is total time_ids.

In [73]:
plt.figure(figsize=(20,10))
dendrogram(Z, labels=df_train_tar.columns, orientation='top', 
           leaf_rotation=90);
ax = plt.gca()
ax.tick_params(axis='x', which='major', labelsize=12)
corr[[3,8,17,20,40,80,81,95,102,110,123]]
list_col = [3,8,17,20,40,80,81,95,102,110]

In [74]:
from tqdm.notebook import tqdm

import numpy as np
import pandas as pd
#pd.set_option('display.max_rows', 500)
#pd.set_option('display.max_columns', 500)
#pd.set_option('display.width', 1000)

import matplotlib.pyplot as plt
import seaborn as sns
import scipy
from scipy.stats import probplot


from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
#https://docs.scipy.org/doc/scipy/reference/tutorial/optimize.html#least-squares-minimization-least-squares
#https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.lsq_linear.html#scipy.optimize.lsq_linear
from scipy.optimize import least_squares, lsq_linear
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')


In [75]:


def Con_regress(X, y, lb, ub): # lb lower bound for variables b
    # constrianted regression
    res = lsq_linear(X, y, bounds=(lb,ub))
    return res.x

def run_regress(X, Y, beta_var): # beta_var is how large we allow beta variates
    k =Y.shape[1]
    Beta = np.eye(X.shape[1],Y.shape[1])
    for i in range(k):
        Beta[:,i] = Con_regress(X,Y[:,i], Beta[:,i]-beta_var, Beta[:,i]+beta_var)
    return Beta

feat = 'rv1' # rv2  rv_0_5   rv_5_10 
list_col = list(range(112)) # [3,8,17,20,40,80,81,95,102,110]
X = features[feat].fillna(value=features[feat].mean()).to_numpy()[:,list_col]
poly = PolynomialFeatures(degree=1)
#X = poly.fit_transform(X)

Y = df_tar.fillna(value=df_tar.mean()).to_numpy()[:,list_col]
X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, test_size=0.2, random_state=142)
B = run_regress(X_train, Y_train, 4)



# random_state =42 || 142
# rv1 train_error:  0.28959 valid_error:  0.30731 || valid_error:  0.35573
# rv2 train_error:  0.27871  valid_error:  0.29147 || valid_error:  0.44310
# rv_0_5 , beta_var=0.5   train_error:  0.32927 valid_error:  0.35617  || beta_var=0.8 valid_error:  0.35551
# rv_5_10 , beta_var=0.5  train_error:  0.30276 valid_error:  0.33233  || valid_error: 0.34606

In [76]:
print('train_error: ', "{:.5f}".format(root_mean_squared_percentage_error(np.matmul(X_train,B), Y_train)),
      'valid_error: ', "{:.5f}".format(root_mean_squared_percentage_error(np.matmul(X_valid,B), Y_valid)))

- We saw when we only use 10 stocks as predictors, the error is reduced from 0.35 to 0.26, this means there are too many redundant stocks when using all stocks which introduced 0.1 variance error without reducing bias.

In [77]:
from sklearn.cross_decomposition import PLSRegression
rmse, rmspe_train, rmspe_valid= [], [], []
max_comp = 15
for comp in range(1,max_comp):
    pls2 = PLSRegression(n_components=comp, scale=False)
    pls2.fit(X_train, Y_train)
    rmse.append(pls2.score(X_train, Y_train))
    Y_train_pred = pls2.predict(X_train)
    Y_valid_pred = pls2.predict(X_valid)
    rmspe_train.append(root_mean_squared_percentage_error(Y_train_pred, Y_train))
    rmspe_valid.append(root_mean_squared_percentage_error(Y_valid_pred, Y_valid))

plt.plot(range(1,max_comp),rmse)
plt.show()   

In [78]:
plt.plot(range(1,max_comp),rmspe_train,label='train')
plt.plot(range(1,max_comp),rmspe_valid,label='valid')
plt.legend(prop={'size': 14})
plt.show()

- Conclusion: There is no magic here. PLS （partial lease square） regression has a comparable predicative power as linear models.  Even with a fancier model, the accuarcy is not improved when adding more components after we have chosen 6 principle components.

## 4.2 correlation preserve regression

In [79]:
def normalize(X):
    X_stat = pd.concat([X.mean().to_frame().transpose(), X.std().to_frame().transpose()], ignore_index=True)
    X = (X-X_stat.loc[0])/X_stat.loc[1]
    return X, X_stat
X, X_stat= normalize(features['rv_5_10']) # n by p : p=112 stocks, n=3830 samples
Y, Y_stat = normalize(df_tar) # target n by p

X_corr =(X.T).dot(X)/3830
Y_corr =(Y.T).dot(Y)/3830
#see_matrix(X.corr()-Y.corr(),0.05)
#see_matrix(X_corr-Y_corr,0.05)
# ||Q* X * Beta - Y ||

## 5. Trade

Trade data represents the aggregation of all individual executed orders for corresponding stocks and time buckets. Size is the sum of the size in each individual order, price is aggregated as a weighted averaged price of all trades and order count is the number of unique trade orders taking place. For trade data, missing seconds_in_bucket implies no trade happening within that one second window.

Trade data files are named as `trade_train.parquet` and `trade_test.parquet`, and they are parquet files partitioned by stock_id column. Partitioned by stock_id means those files can both read as whole or individual stocks. However, reading them as a single file is not easy as they will consume too much memory.

There are 5 columns in every trade data partition. The columns are:

* `time_id` - ID of the time bucket
* `seconds_in_bucket` - Number of seconds passed since the start of the bucket
* `price` - Weighted average price of all executed trades happening in one second
* `size` - Total number of traded shares happening in one second
* `order_count` - Number of unique trade orders happening in one second

Trade data has lots of missing seconds_in_bucket as it is more sparse than order book. Besides, there are some missing time buckets in trade data which means there weren't any trades in that 10 minute window. In addition to that, seconds_in_bucket doesn't necessarily start from 0 in trade data. Trade data can be still reshaped into a fixed size by adding missing time buckets and reindexing time steps to 600 seconds. Missing values can be filled with zeros afterwards since there weren't any trades at those one second windows.

In [80]:
def read_trade_data(df, dataset, stock_id, sort=False, zero_fill=False):
        
    trade_dtypes = {
        'time_id': np.uint16,
        'seconds_in_bucket': np.uint16,
        'price': np.float32,
        'size': np.uint16,
        'order_count': np.uint16
    }

    df_trade = pd.read_parquet(f'../input/optiver-realized-volatility-prediction/trade_{dataset}.parquet/stock_id={stock_id}')
    
    if zero_fill:
        stock_time_buckets = df.loc[df['stock_id'] == stock_id, 'time_id'].reset_index(drop=True)
        missing_time_buckets = stock_time_buckets[~stock_time_buckets.isin(df_trade['time_id'])]
        df_trade = df_trade.merge(missing_time_buckets, how='outer')
    
    if sort:
        df_trade.sort_values(by=['time_id', 'seconds_in_bucket'], inplace=True)
        
    if zero_fill:
        df_trade = df_trade.set_index(['time_id', 'seconds_in_bucket'])
        df_trade = df_trade.reindex(
            pd.MultiIndex.from_product([df_trade.index.levels[0], np.arange(0, 600)], names=['time_id', 'seconds_in_bucket']),
        )
        df_trade.fillna(0, inplace=True)
        df_trade.reset_index(inplace=True)
    
    for column, dtype in trade_dtypes.items():
        df_trade[column] = df_trade[column].astype(dtype)

    return df_trade


Previously, realized volatilities were derived from weighted averaged price of the order book, but they can be derived from trade price as well since it is the weighted averaged price of the trade orders taking place. Realized volatilities derived from trade price are different from the previous ones because trade data is more sparse.

In [81]:
for stock_id in tqdm(sorted(df_train['stock_id'].unique())):
            
    df_trade = read_trade_data(df_train, 'train', stock_id)

    # Realized volatility
    for wap in [1, 2]:
        df_trade['log_return_from_price'] = np.log(df_trade['price'] / df_trade.groupby('time_id')['price'].shift(1))
        df_trade['squared_log_return_from_price'] = df_trade['log_return_from_price'] ** 2
        df_trade['realized_volatility_from_price'] = np.sqrt(df_trade.groupby('time_id')['squared_log_return_from_price'].transform('sum'))
        df_trade.drop(columns=['squared_log_return_from_price'], inplace=True)            
        realized_volatilities = df_trade.groupby('time_id')['realized_volatility_from_price'].first().to_dict()
        df_train.loc[df_train['stock_id'] == stock_id, 'realized_volatility_from_price'] = df_train[df_train['stock_id'] == stock_id]['time_id'].map(realized_volatilities)


Realized volatilies derived from trade price are used as baseline predictions. They score **0.380267** on training set. They can be used as predictors just like the realized volatilities derived from the order book.

In [82]:
realized_volatility_price_rmspe = root_mean_squared_percentage_error(df_train['target'], df_train['realized_volatility_from_price'])
print(f'Realized Volatility from price RMPSE: {realized_volatility_price_rmspe:.6}')

Trade data tells how many shares are traded at what weighted averaged price by how many unique traders for every time step. Those traded shares are the ones listed in the order book at the corresponding time step. Oscillations of trade price, size and order count might be an indicator of realized volatility of the next 10 minute window. Another important thing to consider is sparsity. Sparsity indicates that the stock is calm and there aren't any trades taking place which could be a strong indicator of low realized volatility. However, it may not be a predictor of the next 10 minute windows' realized volatility. The function in the cell below is going to be used for visualizing individual trade data time buckets and trying to find clues about their volatilities. It is important to visualize every sequence with and without zeros in order to understand both oscillations and sparsity.

In [83]:
def visualize_trade_time_bucket(stock_id, time_id):
    
    time_bucket = (df_train['stock_id'] == stock_id) & (df_train['time_id'] == time_id)
    
    target = df_train.loc[time_bucket, 'target'].iloc[0]
    realized_volatility = df_train.loc[time_bucket, 'rv1'].iloc[0]
    df_trade = read_trade_data(df_train, 'train', stock_id, sort=True, zero_fill=True)
    df_trade = df_trade.set_index('seconds_in_bucket')
    
    fig, axes = plt.subplots(figsize=(32, 70), nrows=6)
    
    axes[0].plot(df_trade.loc[(df_trade['time_id'] == time_id) & (df_trade['price'] != 0), 'price'], label='price without zeros', lw=2, linestyle='-', color='tab:blue')
    axes[1].plot(df_trade.loc[(df_trade['time_id'] == time_id), 'price'], label='price with zeros', lw=2, linestyle='--', color='tab:blue')
    axes[2].plot(df_trade.loc[(df_trade['time_id'] == time_id) & (df_trade['price'] != 0), 'size'], label='size without zeros', lw=2, linestyle='-', color='tab:blue')
    axes[3].plot(df_trade.loc[(df_trade['time_id'] == time_id), 'size'], label='size with zeros', lw=2, linestyle='--', color='tab:blue')
    axes[4].plot(df_trade.loc[(df_trade['time_id'] == time_id) & (df_trade['price'] != 0), 'order_count'], label='order_count without zeros', lw=2, linestyle='-', color='tab:blue')
    axes[5].plot(df_trade.loc[(df_trade['time_id'] == time_id), 'order_count'], label='order_count with zeros', lw=2, linestyle='--', color='tab:blue')
    
    for i in range(6):
        axes[i].legend(prop={'size': 18})
        axes[i].tick_params(axis='x', labelsize=20, pad=10)
        axes[i].tick_params(axis='y', labelsize=20, pad=10)
    axes[0].set_ylabel('price', size=20, labelpad=15)
    axes[1].set_ylabel('price', size=20, labelpad=15)
    axes[2].set_ylabel('size', size=20, labelpad=15)
    axes[3].set_ylabel('size', size=20, labelpad=15)
    axes[4].set_ylabel('order_count', size=20, labelpad=15)
    axes[5].set_ylabel('order_count', size=20, labelpad=15)
    
    axes[0].set_title(
        f'Price of stock_id {stock_id} time_id {time_id} without zeros - Current Realized Volatility: {realized_volatility:.6f} - Next 10 minute Realized Volatility: {target:.6f}',
        size=25,
        pad=15
    )
    axes[1].set_title(
        f'Price of stock_id {stock_id} time_id {time_id} with zeros - Current Realized Volatility: {realized_volatility:.6f} - Next 10 minute Realized Volatility: {target:.6f}',
        size=25,
        pad=15
    )
    axes[2].set_title(
        f'Size of stock_id {stock_id} time_id {time_id} without zeros - Current Realized Volatility: {realized_volatility:.6f} - Next 10 minute Realized Volatility: {target:.6f}',
        size=25,
        pad=15
    )
    axes[3].set_title(
        f'Size of stock_id {stock_id} time_id {time_id} with zeros - Current Realized Volatility: {realized_volatility:.6f} - Next 10 minute Realized Volatility: {target:.6f}',
        size=25,
        pad=15
    )
    axes[4].set_title(
        f'Order count of stock_id {stock_id} time_id {time_id} without zeros - Current Realized Volatility: {realized_volatility:.6f} - Next 10 minute Realized Volatility: {target:.6f}',
        size=25,
        pad=15
    )
    axes[5].set_title(
        f'Order count of stock_id {stock_id} time_id {time_id} with zeros - Current Realized Volatility: {realized_volatility:.6f} - Next 10 minute Realized Volatility: {target:.6f}',
        size=25,
        pad=15
    )
    
    plt.show()


As seen before, the most volatile time bucket in next 10 minute window is time **24600** from stock **77**. Current realized volatility of that time bucket is 0.02 which is roughly greater than 99.7% of other current realized volatilities. Trade price without zeros is almost perfectly correlated with weighted averaged price from the order book and trade price with zeros shows that there are trades taking place very frequently. Size shows that there are some extreme amounts of shares traded at certain times by one or multiple traders. Finally, order count shows that there are large number of unique trade orders from time to time regardless of the total size and price.

In [84]:
visualize_trade_time_bucket(stock_id=77, time_id=24600)

As seen before, the least volatile time bucket in next 10 minute window is time **8534** from stock **31**. There are only 4 trades took place in this time bucket which is why it's current realized volatility and next 10 minute window realized volatility is very low. Regardless of the values, sparsity is the obvious reason in this case. 

In [85]:
visualize_trade_time_bucket(stock_id=31, time_id=8534)

There are some time buckets with zero trades exist in trade data. Time **62** from stock **37** is an example of those time buckets. We should use data from book_data to replace the zero.

In [86]:
visualize_trade_time_bucket(stock_id=37, time_id=62)

## 6. Feature Engineering

In [87]:
# To Be Continued